# Welcome to a very busy Week 8 folder

## We have lots to do this week!

We'll move at a faster pace than usual, particularly as you're becoming proficient LLM engineers.


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Especially important this week: pull the latest</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>

In [1]:
# Just one import to start with!!

import modal

# Setting up the modal tokens

The first time you run this, please uncomment the next line and execute it.  
This is the same as running `modal setup` from the command line. It connects with Modal and installs your tokens.

A student on Windows mentioned that on Windows, you might also need to run this command from a command prompt afterwards:  
`modal token new`  
(Thank you Ed B. for that!)

And I've also heard that in some situations, you might need to restart the Kernel of this jupyter notebook after running this. (Kernel menu >> Restart Kernel and Clear Outputs of All Cells).

In [2]:
# Remove the '# ' from the next line and run the cell
!modal setup

The web browser should have opened for you to authenticate and get an API token.
If it didn't, please copy this URL into your web browser manually:

]8;id=988175;https://modal.com/token-flow/tf-4OQWCx9ehoVgotEYIiHbIY\https://modal.com/token-flow/tf-4OQWCx9ehoVgotEYIiHbIY]8;;\

⠙ Waiting for authentication in the web browser
⠧ Waiting for token flow to complete...omplete...
Web authentication finished successfully!
Token is connected to the xykong36 workspace.
Verifying token against https://api.modal.com
Token verified successfully!
⠋ Storing token
Token written to /Users/xiangyu/.modal.toml in profile xykong36.


In [3]:
from hello import app, hello

In [10]:
!modal token new

The web browser should have opened for you to authenticate and get an API token.
If it didn't, please copy this URL into your web browser manually:

]8;id=706003;https://modal.com/token-flow/tf-lfGsLFXreoimWW1H3W4E4A\https://modal.com/token-flow/tf-lfGsLFXreoimWW1H3W4E4A]8;;\

⠋ Waiting for authentication in the web browser
⠙ Waiting for token flow to complete...omplete...
Web authentication finished successfully!
Token is connected to the xykong36 workspace.
Verifying token against https://api.modal.com
Token verified successfully!
⠋ Storing token
Token written to /Users/xiangyu/.modal.toml in profile xykong36.


In [13]:
import os
os.environ["MODAL_TOKEN_ID"] = "ak-KaDVhGcPs78noFTPmU0Ef6"
os.environ["MODAL_TOKEN_SECRET"] = "as-xy7sk1qvTN00oBtIJC6zto"

In [15]:
with app.run():
    reply=hello.local()
reply

'Hello from Charlotte, North Carolina, US!!'

In [20]:
with app.run(show_progress=False):
    reply=hello.remote()
reply

/opt/anaconda3/envs/llms/lib/python3.11/contextlib.py:210: DeprecationError: 2024-11-20: `show_progress=False` is deprecated (and has no effect)
  return await anext(self.gen)


ExecutionError: Function has not been hydrated with the metadata it needs to run on Modal, because the App it is defined on is not running.

In [17]:
import sys

import modal

app = modal.App("example-hello-world")


In [18]:
@app.function()
def f(i):
    if i % 2 == 0:
        print("hello", i)
    else:
        print("world", i, file=sys.stderr)

    return i * i

In [19]:
@app.local_entrypoint()
def main():
    # run the function locally
    print(f.local(1000))

    # run the function remotely on Modal
    print(f.remote(1000))

    # run the function in parallel and remotely on Modal
    total = 0
    for ret in f.map(range(200)):
        total += ret

    print(total)

# Before we move on -

## We need to set your HuggingFace Token as a secret in Modal

1. Go to modal.com, sign in and go to your dashboard
2. Click on Secrets in the nav bar
3. Create new secret, click on Hugging Face, this new secret needs to be called **hf-secret** because that's how we refer to it in the code
4. Fill in your HF_TOKEN where it prompts you

### And now back to business: time to work with Llama

In [ ]:
from llama import app, generate

In [ ]:
with modal.enable_output():
    with app.run():
        result=generate.remote("Life is a mystery, everyone must stand alone, I hear")
result

In [ ]:
import modal
from pricer_ephemeral import app, price

In [ ]:
with modal.enable_output():
    with app.run():
        result=price.remote("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")
result

## Transitioning From Ephemeral Apps to Deployed Apps

From a command line, `modal deploy xxx` will deploy your code as a Deployed App

This is how you could package your AI service behind an API to be used in a Production System.

You can also build REST endpoints easily, although we won't cover that as we'll be calling direct from Python.

In [ ]:
!modal deploy pricer_service

In [ ]:
pricer = modal.Function.lookup("pricer-service", "price")

In [ ]:
pricer.remote("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")

In [ ]:
!modal deploy pricer_service2

In [ ]:
Pricer = modal.Cls.lookup("pricer-service", "Pricer")
pricer = Pricer()
reply = pricer.price.remote("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")
print(reply)

# Optional: Keeping Modal warm

## A way to improve the speed of the Modal pricer service

A student mentioned to me that he was concerned by how slow Modal seems to be. The reason is that Modal puts our service to sleep if we don't use it, and then it takes 2.5 minutes to spin back up.

I've added a utility called `keep_warm.py` that will keep our Modal warm by pinging it every 30 seconds.

To use the utliity, bring up a new Terminal (Mac) or Anaconda prompt (Windows), ensure the environment is activated with `conda activate llms`

Then run: `python keep_warm.py` from within the week8 drectory.

Remember to press ctrl+C or exit the window when you no longer need Modal running.


## And now introducing our Agent class

In [ ]:
from agents.specialist_agent import SpecialistAgent

In [ ]:
agent = SpecialistAgent()
agent.price("iPad Pro 2nd generation")